## Determinar el orden experimentalmente

El experimento es simple: usar el método para calcular la integral de una función conocida, usando dos valores para el paso $h$ diferentes y comparar ambos resultados.  

https://www.youtube.com/watch?v=6O9D6am_RK4&list=PLMsYJgjgZE8iBpOBZEsS8PuwNBkwMcjix&index=60

## Teorema Fundamental del Cálculo

https://es.wikipedia.org/wiki/Teorema_fundamental_del_c%C3%A1lculo



Dada una función f(x) integrable en el intervalo $[a,b]$ y sea F(x) cualquier función primitiva de f, es decir F '(x) = f(x). Entonces:

$
\int_{a}^{b}{f(x) d(x)} = F(b) − F(a)
$

Dado un PVI:

$
\begin{cases}
y'(t) = f(t) \\
y(t_0) = y_0
\end{cases}
$

Vemos que calcular la solución $\hat{y}(t)$ es lo mismo que integrar numéricamente $f(t)$. Además, pongamos que conocemos la solución exacta $y(t)$. Podemos compararla con la numérica. 

## Ejemplo

$
\begin{cases}
y'(t) = \frac{1}{(1+t)^2} \\
y(0) = 0
\end{cases}
$

Sabemos que $y(t) = -\frac{1}{1+t} + 1$

Calculamos por ejemplo la integral entre $t=0$ y $t=1$, o lo que es lo mismo, $y(1) - y(0)$, de manera analítica: 

$
\int_0^1{\frac{1}{(1+t)^2}} = y(1) - y(0) = [-\frac{1}{1+1}+1]-[-\frac{1}{1+0}+1] = \frac{1}{2}
$

De hecho el valor inicial y(0) = 0 está escogido para simplificar lo anterior: solo necesitamos calcular $y(1)$

Tambien calculamos $y(1)$ de manera numérica y nos debería de salir un valor próximo a $1/2$. Probamos usando varios valores de $h$: cuanto más pequeño sea $h$ menor será el error cometido por el método. 

>Precisamente, **el orden del método indica cuánto se reduce el error al reducir h**.

El ratio entre los dos errores obtenidos con dos pasos h distintos es equivalente al orden del método.

Por ejemplo si el método tiene orden $O(h^3)$ y calculamos dos soluciones: una con paso h y otra con paso h/2 los errores quedan $e_{h} \approx O(h^3)$ y $e_{h/2} \approx O((\frac{h}{2})^3)$. Eso quiere decir que:

>$\frac{e_1}{e_2} \approx O(\frac{h^n}{(h/2)^n}) = O((\frac{h}{(h/2)})^n) = O(2^n) \quad \rightarrow  \quad \log_2{\frac{e_1}{e_2}} \approx O(n)$

## Preguntas: 

**1. ¿Qué f(t) es mejor usar? ¿hay alguna que se comporta mejor como modelo para "detectar" el orden de los métodos?**

**2. ¿Es mejor avanzar un solo paso o varios? En un caso estaríamos estimando el error local y en el otro el global ¿Qué relación hay entre ellos?**

In [68]:
import numpy as np
import math

def euler(f, t0=0, t_end=10, y0=0, h=0.1):
    tn, yn = t0, y0
    ts, ys = [t0], [y0]

    assert len(f(t0, y0)) == len(y0)

    for _ in range(int((t_end-t0)/h)):
        yn = yn + h*f(tn, yn)
        tn = tn + h
        ts.append(tn)
        ys.append(yn)
    return ts, np.array(ys)


def heun(f, t0=0, t_end=10, y0=0, h=0.1):
    tn, yn = t0, y0
    ts, ys = [t0], [y0]

    assert len(f(t0, y0)) == len(y0)

    for _ in range(int((t_end-t0)/h)):
        yn = yn + h*0.5*(f(tn, yn) + f(tn + h, yn + h*f(tn,yn)))
        tn = tn + h
        ts.append(tn)
        ys.append(yn)
    return ts, np.array(ys)


def rungekutta(f, t0=0, t_end=10, y0=0, h=0.1):
    tn, yn = t0, y0
    ts, ys = [t0], [y0]

    assert len(f(t0, y0)) == len(y0)

    for _ in range(int((t_end-t0)/h)):
        k1 = f(tn, yn)
        k2 = f(tn+(h/2), yn+(h/2)*k1)
        k3 = f(tn+(h/2), yn+(h/2)*k2)
        k4 = f(tn+h, yn+h*k3)
        yn = yn + h*(1/6)*(k1+2*k2+2*k3+k4)
        tn = tn + h
        ts.append(tn)
        ys.append(yn)
    return ts, np.array(ys)


def method_order(method, **kwargs):
    f = lambda t, y: np.array([1/(1+t)**2])
    exact = 0.5
    h = 0.1
    T,Y1 = method(f, t0 = 0, t_end = 1, y0 = [0], h = h, **kwargs)
    T,Y2 = method(f, t0 = 0, t_end = 1, y0 = [0], h = h/2, **kwargs)
    ratio = (Y1[-1][0] - exact) / (Y2[-1][0] - exact)
    p = round(math.log2(ratio))
    print(f"ratio={ratio}, log(ratio)={math.log2(ratio)}, orden={p}")
    return p


method_order(euler)
method_order(heun)
method_order(rungekutta)

ratio=2.0380008996874377, log(ratio)=1.0271546883900458, orden=1
ratio=3.9934133712889754, log(ratio)=1.997622417786525, orden=2
ratio=15.89199980684104, log(ratio)=3.990228775890807, orden=4


4

## Probando distintas funciones

In [71]:
def method_order(method, f, y, h=0.1, **kwargs):
    t_end = 1
    exact = y(t_end)
    T,Y1 = method(f, t0 = 0, t_end = t_end, y0 = [0], h = h, **kwargs)
    T,Y2 = method(f, t0 = 0, t_end = t_end, y0 = [0], h = h/2, **kwargs)
    ratio = (Y1[-1][0] - exact) / (Y2[-1][0] - exact)
    p = round(math.log2(ratio))
    print(f"exact={exact} ratio={ratio}, log(ratio)={math.log2(ratio)}, orden={p}")
    return p


f = lambda t, y: np.array([1/(1+t)**2])
y = lambda t: np.array([-(1/(1+t))+1])

f = lambda t, y: np.array([t])
y = lambda t: np.array([t**2])

f = lambda t, y: np.array([np.exp(t)])
y = lambda t: np.array([np.exp(t)-1])

method_order(euler, f, y)
method_order(heun, f, y)
method_order(rungekutta, f, y)

np.exp(1)-1

exact=[1.71828183] ratio=[1.98319818], log(ratio)=0.9878288553726013, orden=1
exact=[1.71828183] ratio=[3.99950013], log(ratio)=1.999819697999071, orden=2
exact=[1.71828183] ratio=[15.99642985], log(ratio)=3.999678048789134, orden=4


1.718281828459045